In [1]:
# Author: Tiago Tamagusko (tamagusko@gmail.com)
# Version: 3.0 (2023-09-16)

In [2]:
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

In [3]:
# Data

# Structure
AGE = pd.read_csv('raw/age.csv')
# THICKNESS = pd.read_csv('raw/repr_thickness.csv')
SN = pd.read_csv('raw/sn.csv') # structure number
# Traffic
AADTT = pd.read_csv('raw/aadtt.csv')
# Climate
PRECIPITATION = pd.read_csv('raw/precipitation.csv')
TEMPERATURE = pd.read_csv('raw/temperature.csv')
# Performance
IRI = pd.read_csv('raw/iri.csv')

In [4]:
# Fixing column ' ' in PRECIPITATION dataset
PRECIPITATION = PRECIPITATION.copy()
PRECIPITATION.rename(columns={' ': 'STATE_CODE'}, inplace=True)

In [5]:
# Cleaning data
AGE = AGE[['STATE_CODE', 'SHRP_ID', 'TRAFFIC_OPEN_DATE']]
AGE['TRAFFIC_OPEN_DATE'] = pd.DatetimeIndex(AGE['TRAFFIC_OPEN_DATE']).year
AGE['TRAFFIC_OPEN_DATE'] = AGE['TRAFFIC_OPEN_DATE'].astype("Int64")
# THICKNESS = THICKNESS[['STATE_CODE', 'SHRP_ID', 'REPR_THICKNESS', 'LAYER_TYPE']]
SN = SN[['STATE_CODE', 'SHRP_ID', 'SN_VALUE']]
AADTT = AADTT[['STATE_CODE', 'SHRP_ID', 'YEAR', 'AADTT_ALL_TRUCKS_TREND']]
PRECIPITATION = PRECIPITATION[['STATE_CODE', 'SHRP_ID', 'YEAR', 'TOTAL_ANN_PRECIP']]
TEMPERATURE = TEMPERATURE[['STATE_CODE', 'SHRP_ID', 'YEAR', 'MEAN_ANN_TEMP_AVG']]
IRI = IRI[['STATE_CODE', 'SHRP_ID', 'VISIT_DATE', 'MRI']]
IRI['VISIT_DATE'] = pd.DatetimeIndex(IRI['VISIT_DATE']).year
IRI.rename(columns={'VISIT_DATE': 'YEAR'}, inplace=True)

In [6]:
# Sum the AC layers for each section.
# filtered_thickness = THICKNESS[THICKNESS['LAYER_TYPE'] == 'AC']
# AC_THICKNESS_SUM = filtered_thickness.groupby(['STATE_CODE', 'SHRP_ID'])['REPR_THICKNESS'].sum().reset_index()
# AC_THICKNESS_SUM.rename(columns={'REPR_THICKNESS': 'AC_THICKNESS'}, inplace=True)

In [7]:
average_IRI = IRI.groupby(['STATE_CODE', 'SHRP_ID', 'YEAR'])['MRI'].mean().reset_index()

In [8]:
# Define a function to get the first value of a series
def get_first_value(series):
    return series.iloc[0]

# Apply the function on the 'MRI' column grouped by the two specified columns
average_IRI['INITIAL_IRI'] = average_IRI.groupby(['STATE_CODE', 'SHRP_ID'])['MRI'].transform(get_first_value)

In [9]:
average_IRI.head()

,STATE_CODE,SHRP_ID,YEAR,MRI,INITIAL_IRI
0,1,1021,1990,0.9694,0.9694
1,1,1021,1992,0.9484,0.9694
2,1,1021,1994,0.9882,0.9694
3,1,1021,1995,1.0054,0.9694
4,1,1021,1999,1.1644,0.9694


In [10]:
# Creating a unified dataset
DATA = AADTT.copy()
DATA = pd.merge(DATA, AGE, on=['SHRP_ID', 'STATE_CODE'], how='right')
# DATA = pd.merge(DATA, AC_THICKNESS_SUM, on=['SHRP_ID', 'STATE_CODE'], how='right')
DATA = pd.merge(DATA, SN, on=['SHRP_ID', 'STATE_CODE'], how='right')
DATA = pd.merge(DATA, PRECIPITATION, on=['SHRP_ID', 'STATE_CODE', 'YEAR'], how='right')
DATA = pd.merge(DATA, TEMPERATURE, on=['SHRP_ID', 'STATE_CODE', 'YEAR'], how='right')
DATA = pd.merge(DATA, average_IRI, on=['SHRP_ID', 'STATE_CODE', 'YEAR'], how='right')
DATA.shape

(395, 10)

In [11]:
# STATION_ID = STATE_CODE _SHRP_ID
DATA['STATION_ID'] = DATA['STATE_CODE'].astype(str) + "_" + DATA['SHRP_ID'].astype(str)
DATA = DATA.drop(['STATE_CODE', 'SHRP_ID'], axis=1)

In [12]:
# Sort dataset
DATA = DATA.sort_values(['YEAR'], ascending=[True])

In [13]:
DATA.columns

Index(['YEAR', 'AADTT_ALL_TRUCKS_TREND', 'TRAFFIC_OPEN_DATE', 'SN_VALUE',
       'TOTAL_ANN_PRECIP', 'MEAN_ANN_TEMP_AVG', 'MRI', 'INITIAL_IRI',
       'STATION_ID'],
      dtype='object')

In [14]:
# Rename columns
DATA = DATA.rename({'AADTT_ALL_TRUCKS_TREND': 'AADTT',
                    'AC_THICKNESS': 'THICKNESS',
                    'SN_VALUE': 'SN',
                    'MRI': 'IRI',
                    'TOTAL_ANN_PRECIP': 'PRECIPITATION',
                    'MEAN_ANN_TEMP_AVG': 'TEMPERATURE',
                    }, axis=1)

In [15]:
DATA.head()

,YEAR,AADTT,TRAFFIC_OPEN_DATE,SN,PRECIPITATION,TEMPERATURE,IRI,INITIAL_IRI,STATION_ID
244,1989,1190.0,1985,6.1,1372.800049,12.000000,0.8984,0.8984,34_1638
136,1989,490.0,1985,3.3,260.799988,6.100000,1.2738,1.2738,16_1021
233,1989,151.0,1982,4.3,634.200012,6.000000,1.0314,1.0314,30_1001
277,1989,195.0,1984,3.2,1546.099976,16.200001,1.0448,1.0448,37_1030
166,1989,1690.0,1985,8.3,1125.199951,7.200000,0.7238,0.7238,23_1012


In [16]:
DATA.describe().T

,count,mean,std,min,25%,50%,75%,max
YEAR,395.0,1995.612658,4.974998,1989.0,1992.0,1994.0,1999.0,2016.0
AADTT,395.0,471.632911,539.745625,11.0,156.0,278.0,536.0,3126.0
TRAFFIC_OPEN_DATE,395.0,1980.321519,5.449856,1970.0,1975.0,1983.0,1984.0,1989.0
SN,395.0,4.627089,1.554672,2.0,3.6,4.2,5.7,8.3
PRECIPITATION,395.0,1006.025572,471.496033,92.199997,631.899994,1059.800049,1363.850036,2091.0
TEMPERATURE,395.0,14.613418,6.046667,2.7,8.5,15.8,18.299999,25.9
IRI,395.0,1.370352,0.598253,0.6208,0.9203,1.176,1.6814,4.0048
INITIAL_IRI,395.0,1.214235,0.448197,0.6406,0.8882,1.0502,1.4934,2.515


In [17]:
# verify data
DATA.info()

<class 'pandas.core.frame.DataFrame'>
Index: 395 entries, 244 to 220
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   YEAR               395 non-null    int64  
 1   AADTT              395 non-null    float64
 2   TRAFFIC_OPEN_DATE  395 non-null    Int64  
 3   SN                 395 non-null    float64
 4   PRECIPITATION      395 non-null    float64
 5   TEMPERATURE        395 non-null    float64
 6   IRI                395 non-null    float64
 7   INITIAL_IRI        395 non-null    float64
 8   STATION_ID         395 non-null    object 
dtypes: Int64(1), float64(6), int64(1), object(1)
memory usage: 31.2+ KB


In [18]:
# Save data
DATA.to_csv('processed/ltpp_data.csv', index=None, header=True)